<a href="https://colab.research.google.com/github/JacobyNevada/ds_belhard/blob/main/hw2/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data_loader.py

import pandas as pd
from google.colab import drive
import os

class Data_Loader:
    def data_loading(self, file_link, category, numeric_features, categorical_features):
      try:
        drive.mount('/content/drive')
        file_path = f"/content/drive/{file_link}"

        # Загрузка CSV файла
        dataset = pd.read_csv(file_path)

        if not os.path.exists(file_path):
          raise FileNotFoundError(f"Файл {file_path} не найден")

        # Проверяем, что файл не пустой
        if os.path.getsize(file_path) == 0:
          raise ValueError(f"Файл {file_path} пустой")

        return dataset[numeric_features + categorical_features + [category]]

      except FileNotFoundError as e:
        print(f"Ошибка: {e}")
        print("Проверьте наличие файла в Google Drive")
        return None

      except ValueError as e:
        print(f"Ошибка данных: {e}")
        return None

      except Exception as e:
        print(f"Неожиданная ошибка: {e}")
        return None

In [ ]:
# data_procesor.py

import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

class Data_Processor:

  def data_processing(self, dataset, category, numeric_features, categorical_features):
    X = dataset.drop(columns=[category])
    y = dataset[category]

    # Создание препроцессора
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(drop='first')

    preprocessor = ColumnTransformer(
      transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Применение препроцессора к данным
    X_processed = preprocessor.fit_transform(X)
    return X_processed, y, preprocessor

In [22]:
!git clone https://github.com/JacobyNevada/ds_belhard.git

# 2. Переходим в папку hw2
%cd /content/ds_belhard/hw2

# 3. Добавляем пути для импорта
import sys
sys.path.append('/content/ds_belhard')      # Корень репозитория
sys.path.append('/content/ds_belhard/hw2')  # Папка hw2

# 4. Проверяем какие файлы есть в папке hw2
!ls -la /content/ds_belhard/hw2/

Cloning into 'ds_belhard'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 883.84 KiB | 6.55 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/ds_belhard/hw2
total 1144
drwxr-xr-x 3 root root    4096 Sep 15 18:27 .
drwxr-xr-x 6 root root    4096 Sep 15 18:24 ..
-rw-r--r-- 1 root root    1671 Sep 15 18:24 data_procesor.py
drwxr-xr-x 6 root root    4096 Sep 15 18:27 ds_belhard
-rw-r--r-- 1 root root       1 Sep 15 18:24 .gitkeep
-rw-r--r-- 1 root root 1144997 Sep 15 18:24 hw2.ipynb
-rw-r--r-- 1 root root    1410 Sep 15 18:24 Readme.md


In [20]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt

from data_loader import Data_Loader

####################################
# data_loader.py

'''import pandas as pd
from google.colab import drive
import os

class Data_Loader:

  def data_loading(file_link, category, numeric_features, categorical_features):
    try:
      drive.mount('/content/drive')
      file_path = f"/content/drive/{file_link}"

      # Загрузка CSV файла
      dataset = pd.read_csv(file_path)

      if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {file_path} не найден")

      # Проверяем, что файл не пустой
      if os.path.getsize(file_path) == 0:
        raise ValueError(f"Файл {file_path} пустой")

      imputer = SimpleImputer(strategy='mean')

      # Выбираем только числовые колонки для импутации
      numeric_columns = numeric_features + [category]

      # Применяем импутер
      dataset[numeric_columns] = imputer.fit_transform(dataset[numeric_columns])

      imputer = SimpleImputer(strategy='most_frequent')

      # Применяем к строковым колонкам
      string_columns = categorical_features
      dataset[string_columns] = imputer.fit_transform(dataset[string_columns])

      return dataset[numeric_features + categorical_features + [category]]

    except FileNotFoundError as e:
      print(f"Ошибка: {e}")
      print("Проверьте наличие файла в Google Drive")
      return None

    except ValueError as e:
      print(f"Ошибка данных: {e}")
      return None

    except Exception as e:
      print(f"Неожиданная ошибка: {e}")
      return None'''


'''def data_processing(dataset, category, numeric_features, categorical_features):
  X = dataset.drop(columns=[category])
  y = dataset[category]

  # Создание препроцессора
  numeric_transformer = StandardScaler()
  categorical_transformer = OneHotEncoder(drop='first')

  preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numeric_features),
          ('cat', categorical_transformer, categorical_features)
      ])

  # Применение препроцессора к данным
  X_processed = preprocessor.fit_transform(X)
  return X_processed, y, preprocessor'''

###############################
# data_procesor.py

import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

class Data_Processor:

  def data_processing(dataset, category, numeric_features, categorical_features):
    X = dataset.drop(columns=[category])
    y = dataset[category]

    # Создание препроцессора
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(drop='first')

    preprocessor = ColumnTransformer(
      transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Применение препроцессора к данным
    X_processed = preprocessor.fit_transform(X)
    return X_processed, y, preprocessor

###############################
# data_trainer.py

import pandas as pd

from sklearn.linear_model import LogisticRegression

class Data_Trainer:
  def train_model(X, y):
    model = LogisticRegression(solver='lbfgs', max_iter=1000)
    model.fit(X, y)
    return model


  def predict(model, X):
    return model.predict(X)


  def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(report)
    #return accuracy, report


###############################
# data_visualiser.py

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

class Data_Visualiser:
  def visualise_pairplot(dataset, category):
    sample_data = dataset.head(50)
    # Построение парных графиков при помощи sns
    sns.pairplot(sample_data, hue=category)
    plt.show()


  def visualise_histogram(dataset):
    # Установка стиля Seaborn для красивых графиков
    sns.set(style="whitegrid")

    # Создание гистограмм для каждой числовой переменной
    dataset.hist(bins=20, figsize=(15, 10), color='skyblue', edgecolor='black')

    # Добавление названий для каждого графика и осей
    for ax in plt.gcf().get_axes():
      ax.set_xlabel('Значение')
      ax.set_ylabel('Частота')
      #ax.set_title(ax.get_title().replace('wine_class', 'Класс вина'))

    # Регулировка макета для предотвращения наложения подписей
    plt.tight_layout()

    # Показать график
    plt.show()


  def visualise_heatmap(dataset):
    sns.set(style="white")

    # Расчет корреляционной матрицы только для числовых данных
    numeric_df = dataset.select_dtypes(include=[np.number])  # Исключаем нечисловые столбцы
    corr = numeric_df.corr()

    # Маска для отображения только нижней треугольной части матрицы (опционально)
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Настройка цветовой палитры
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # Создание тепловой карты
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

    # Добавление заголовка
    plt.title('Тепловая карта корреляций', fontsize=20)

    # Показать график
    plt.show()

  def visualise_boxplot(dataset):
    sns.set(style="whitegrid")

    # Создаем ящики с усами для каждой колонки в DataFrame
    plt.figure(figsize=(12, 10))

    # Перебираем каждый числовой столбец и создаем для него ящик с усами
    for index, column in enumerate(dataset.select_dtypes(include=[np.number]).columns):
      plt.subplot((len(dataset.columns) // 3) + 1, 3, index + 1)
      sns.boxplot(y=dataset[column])

    plt.tight_layout()
    plt.show()

  def visualise_lineplot(dataset, numeric_features):
    sns.set(style="whitegrid")

    # Создаем сетку графиков
    fig, axes = plt.subplots(len(numeric_features), 1, figsize=(15, 20))

    # Для случая с одним графиком (если нужен только один столбец)
    if len(numeric_features) == 1:
        axes = [axes]

    datahead = dataset.head(20)

    # Строим line plot для каждого числового столбца
    for i, column in enumerate(numeric_features):
        axes[i].plot(datahead.index, datahead[column], marker='o', linestyle='-',
                    linewidth=2, markersize=3, alpha=0.7)
        axes[i].set_xlabel('Индекс наблюдения')
        axes[i].set_ylabel('Значение')
        axes[i].set_title(f'Линейный график: {column}')
        axes[i].grid(True, alpha=0.3)

    # Регулировка макета для предотвращения наложения подписей
    plt.tight_layout()

    # Показать график
    plt.show()


  def visualise_all_types(dataset, numeric_features, category):
    visualise_pairplot(dataset, category)
    visualise_histogram(dataset)
    visualise_heatmap(dataset)
    visualise_boxplot(dataset)
    visualise_lineplot(dataset, numeric_features)

###############################
# data_informer.py

import pandas as pd

class Data_Informer:
  def show_dataset(dataset):
    print(f"\nDataset head:\n{dataset}\n")


  def show_info(dataset):
    print(f"\nDataset info:\n{dataset.info()}\n")


  def show_describe(dataset):
    print(f"\nDataset describe:\n{dataset.describe()}\n")


  def show_types(dataset):
    print(f"\nDataset types:\n{dataset.dtypes}\n")


  def show_null_values(dataset):
    print(f"\nDataset null values:\n{dataset.isnull().sum()}\n")


  def show_nan_values(dataset):
    print(f"\nDataset NaN values:\n{dataset.isna().sum()}\n")


  def show_all_info(dataset):
    show_dataset(dataset)
    show_info(dataset)
    show_describe(dataset)
    show_types(dataset)
    show_null_values(dataset)
    show_nan_values(dataset)




################################

# https://www.kaggle.com/datasets/nabihazahid/spotify-dataset-for-churn-analysis
file_link = 'MyDrive/Colab Notebooks/spotify_churn_dataset.csv'

# Определение числовых и категориальных признаков
numeric_features = ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week', 'offline_listening']
categorical_features = ['gender', 'country', 'device_type', 'subscription_type']
category = 'is_churned'

# Загружаем информацию
dataset = Data_Loader.data_loading(file_link, category, numeric_features, categorical_features)

# Показать всю информацию о датасете
Data_Informer.show_all_info(dataset)

# Обработка информации
X, y, preprocessor = Data_Processor.data_processing(dataset, category, numeric_features, categorical_features)

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучение модели
model = Data_Trainer.train_model(X_train, y_train)

# Предсказание на тестовых данных
y_pred = Data_Trainer.predict(model, X_test)

# Оценка модели
Data_Trainer.evaluate_model(y_test, y_pred)

# Показать все визуализации
Data_Visualiser.visualise_all_types(dataset, numeric_features, category)

TypeError: Data_Loader.data_loading() missing 1 required positional argument: 'categorical_features'